In [106]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import pipeline 
from sklearn.svm import SVC
import time

In [107]:
df = pd.read_csv('heart.csv')
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [108]:
#feature engineer
df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [109]:
np.random.seed(0)

In [110]:
def data_enhancement(df):
    
    data = df
    
    for output in data['output'].unique():
        output_data       =  data[data['output'] == output]
        trtbps_std = output_data['trtbps'].std()
        chol_std = output_data['chol'].std()
        thalachh_std = output_data['thalachh'].std()
       
        
        for i in data[data['output'] == output].index:
            if np.random.randint(2) == 1:
                data['trtbps'].values[i] += trtbps_std/10
            else:
                data['trtbps'].values[i] -= trtbps_std/10
                
            if np.random.randint(2) == 1:
                data['chol'].values[i] += chol_std/10
            else:
                data['chol'].values[i] -= chol_std/10
                
            if np.random.randint(2) == 1:
                data['thalachh'].values[i] += thalachh_std/10
            else:
                data['thalachh'].values[i] -= thalachh_std/10

    return data

In [111]:
new_data = data_enhancement(df)
new_data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,143,238,1,0,151,0,2.3,0,0,1,1
1,37,1,2,128,255,0,1,188,0,3.5,0,0,2,1
2,41,0,1,131,209,0,0,173,0,1.4,2,0,2,1
3,56,1,1,121,241,0,1,176,0,0.8,2,0,2,1
4,57,0,0,118,359,0,1,161,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,138,245,0,1,120,1,0.2,1,0,3,0
299,45,1,3,111,259,0,1,134,0,1.2,1,0,3,0
300,68,1,0,145,197,1,1,143,0,3.4,1,2,3,0
301,57,1,0,128,126,0,1,112,1,1.2,1,1,3,0


In [112]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: output, Length: 303, dtype: int64

In [113]:
scaler_Models = pipeline.Pipeline(steps=[('scaling' , StandardScaler())])

In [114]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
  "Extra Trees":ExtraTreesClassifier(random_state=0),
  "Random Forest":RandomForestClassifier(random_state=0),
  "AdaBoost":AdaBoostClassifier(random_state=0),
  "Skl GBM": GradientBoostingClassifier(random_state=0),
  "Skl HistGBM":HistGradientBoostingClassifier(random_state=0),
  "XGBoost": XGBClassifier(random_state=0),
  "LightGBM":LGBMClassifier(random_state=0),
  "CatBoost": CatBoostClassifier(random_state=0),
  "Svm":      SVC(random_state = 0)}
tree_classifiers = {name: pipeline.make_pipeline(scaler_Models, model) for name, model in tree_classifiers.items()}  
 


In [115]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0, test_size=0.2)



results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    # tree_prepro.fit(x_train)
    # X_train_transformed = tree_prepro.transform(x_train)
    # X_train_transformed = pd.DataFrame(X_train_transformed, columns=list(num_vars) + list(cat_vars))
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_test)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              




results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
# results_ord.index += 1 
# results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')
results

C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":

[22:51:41] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,


Learning rate set to 0.005621
0:	learn: 0.6885641	total: 4.14ms	remaining: 4.14s
1:	learn: 0.6838767	total: 7.66ms	remaining: 3.82s
2:	learn: 0.6800161	total: 10.3ms	remaining: 3.42s
3:	learn: 0.6755699	total: 13.1ms	remaining: 3.25s
4:	learn: 0.6711260	total: 15.8ms	remaining: 3.15s
5:	learn: 0.6671142	total: 18ms	remaining: 2.98s
6:	learn: 0.6637292	total: 19.6ms	remaining: 2.78s
7:	learn: 0.6601031	total: 21.5ms	remaining: 2.67s
8:	learn: 0.6558187	total: 23.6ms	remaining: 2.6s
9:	learn: 0.6516547	total: 25.4ms	remaining: 2.51s
10:	learn: 0.6484048	total: 28.1ms	remaining: 2.53s
11:	learn: 0.6439559	total: 30.1ms	remaining: 2.48s
12:	learn: 0.6409952	total: 40.4ms	remaining: 3.07s
13:	learn: 0.6366159	total: 42.7ms	remaining: 3.01s
14:	learn: 0.6330914	total: 44.4ms	remaining: 2.92s
15:	learn: 0.6295922	total: 45.7ms	remaining: 2.81s
16:	learn: 0.6255725	total: 47ms	remaining: 2.72s
17:	learn: 0.6219149	total: 48.6ms	remaining: 2.65s
18:	learn: 0.6181123	total: 50.2ms	remaining: 2.5

C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\485198881.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,


,Model,Accuracy,Bal Acc.,Time
0,Decision Tree,75.409836,76.034858,0.005980
1,Extra Trees,83.606557,83.387800,0.116687
2,Random Forest,86.885246,87.091503,0.144609
3,AdaBoost,86.885246,87.091503,0.068777
4,Skl GBM,81.967213,82.298475,0.074857
5,Skl HistGBM,81.967213,81.917211,0.324129
6,XGBoost,80.327869,80.446623,0.108712
7,LightGBM,83.606557,83.387800,0.048868
8,CatBoost,83.606557,83.387800,1.569216
9,Svm,86.885246,85.947712,0.007976


In [116]:
extra_sample = new_data.sample(new_data.shape[0] // 4)
x_train_enh = pd.concat([x_train, extra_sample.drop(['output'], axis=1 ) ])
y_train_ehn = pd.concat([y_train, extra_sample['output'] ])

In [117]:
# scaler_2 = StandardScaler()
# x_train_enh = scaler_2.fit_transform(x_train_enh)
# x_test = scaler_2.transform(x_test)



results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()

    model.fit(x_train_enh,y_train_ehn)
   
    pred = model.predict(x_test)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              




results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
# results_ord.index += 1 
# results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')
results

C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    

[22:51:44] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,


Learning rate set to 0.006308
0:	learn: 0.6880177	total: 1.52ms	remaining: 1.51s
1:	learn: 0.6829580	total: 3.26ms	remaining: 1.63s
2:	learn: 0.6781583	total: 4.61ms	remaining: 1.53s
3:	learn: 0.6729274	total: 6.05ms	remaining: 1.51s
4:	learn: 0.6689475	total: 7.44ms	remaining: 1.48s
5:	learn: 0.6645438	total: 8.8ms	remaining: 1.46s
6:	learn: 0.6594881	total: 10.3ms	remaining: 1.46s
7:	learn: 0.6539352	total: 11.6ms	remaining: 1.44s
8:	learn: 0.6490164	total: 13ms	remaining: 1.44s
9:	learn: 0.6447617	total: 14.2ms	remaining: 1.41s
10:	learn: 0.6403053	total: 15.6ms	remaining: 1.4s
11:	learn: 0.6355263	total: 17ms	remaining: 1.4s
12:	learn: 0.6312023	total: 18.8ms	remaining: 1.43s
13:	learn: 0.6262382	total: 20.4ms	remaining: 1.43s
14:	learn: 0.6224726	total: 21.6ms	remaining: 1.42s
15:	learn: 0.6182759	total: 23ms	remaining: 1.41s
16:	learn: 0.6141935	total: 24.8ms	remaining: 1.44s
17:	learn: 0.6099959	total: 26.2ms	remaining: 1.43s
18:	learn: 0.6060152	total: 27.6ms	remaining: 1.43s
1

C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\omolara\AppData\Local\Temp\ipykernel_22236\25591132.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,


,Model,Accuracy,Bal Acc.,Time
0,Decision Tree,83.606557,83.769063,0.009023
1,Extra Trees,88.524590,88.180828,0.128512
2,Random Forest,88.524590,88.562092,0.141850
3,AdaBoost,90.163934,90.032680,0.081776
4,Skl GBM,90.163934,90.795207,0.113735
5,Skl HistGBM,85.245902,85.239651,0.373769
6,XGBoost,86.885246,86.710240,0.111611
7,LightGBM,85.245902,85.239651,0.047794
8,CatBoost,86.885246,87.091503,1.830660
9,Svm,88.524590,88.180828,0.014960


In [118]:
# extra_sample = new_data.sample(new_data.shape[0] // 4)
# new_df= pd.concat([df, extra_sample])
# new_df.shape

In [119]:
# x = new_df.iloc[:,:-1]
# y = new_df.iloc[:,-1]

In [120]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(x_train)
# X_test = scaler.transform(x_test)

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=0, max_depth=4, n_estimators=200)

# from sklearn.metrics import  accuracy_score
# rf.fit(X_train,y_train)
# pred =  rf.predict(X_test)
# acc = accuracy_score(y_test, pred)

# acc
